In [25]:
# train.py

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models
from prepare_data import create_dataloaders

# ----------------------
# CONFIGURATION
# ----------------------
NUM_CLASSES = 29  # Update if needed
BATCH_SIZE = 32
NUM_EPOCHS = 20
LEARNING_RATE = 1e-4
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


# ----------------------
# STEP 1: Load Data
# ----------------------
train_loader, val_loader, test_loader = create_dataloaders(batch_size=BATCH_SIZE)



Datasets ready: 1212 train, 335 val, 205 test samples.


In [26]:
# ----------------------
# STEP 2: Load Model
# ----------------------
model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)



In [27]:
# Replace the final layer
model.fc = nn.Linear(model.fc.in_features, NUM_CLASSES)
model = model.to(DEVICE)
model


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [28]:
# ----------------------
# STEP 3: Define Loss and Optimizer
# ----------------------
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)



In [29]:
# ----------------------
# STEP 4: Training Loop
# ----------------------
def train(model, train_loader, val_loader, criterion, optimizer, num_epochs=NUM_EPOCHS):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

        train_loss = running_loss / total
        train_acc = 100. * correct / total

        val_loss, val_acc = evaluate(model, val_loader, criterion)

        print(f"Epoch [{epoch+1}/{num_epochs}] | Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}% | Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.2f}%")

# ----------------------
# STEP 5: Evaluation Function
# ----------------------
def evaluate(model, loader, criterion):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * inputs.size(0)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    loss = running_loss / total
    acc = 100. * correct / total
    return loss, acc





In [30]:
# ----------------------
# STEP 6: Start Training
# ----------------------
if __name__ == '__main__':
    train(model, train_loader, val_loader, criterion, optimizer, num_epochs=15)

    # Save the model
    torch.save(model.state_dict(), 'resnet50_marburglens.pth')
    print("Training complete and model saved!")

Epoch [1/15] | Train Loss: 2.4510 | Train Acc: 38.86% | Val Loss: 1.1011 | Val Acc: 72.54%
Epoch [2/15] | Train Loss: 1.0296 | Train Acc: 76.32% | Val Loss: 0.4922 | Val Acc: 91.04%
Epoch [3/15] | Train Loss: 0.5207 | Train Acc: 89.11% | Val Loss: 0.2220 | Val Acc: 95.22%
Epoch [4/15] | Train Loss: 0.3584 | Train Acc: 91.42% | Val Loss: 0.1828 | Val Acc: 95.52%
Epoch [5/15] | Train Loss: 0.2711 | Train Acc: 94.22% | Val Loss: 0.0897 | Val Acc: 98.51%
Epoch [6/15] | Train Loss: 0.2010 | Train Acc: 95.87% | Val Loss: 0.0926 | Val Acc: 98.21%
Epoch [7/15] | Train Loss: 0.1887 | Train Acc: 95.71% | Val Loss: 0.0951 | Val Acc: 98.21%
Epoch [8/15] | Train Loss: 0.1788 | Train Acc: 95.46% | Val Loss: 0.1044 | Val Acc: 97.91%
Epoch [9/15] | Train Loss: 0.1598 | Train Acc: 96.12% | Val Loss: 0.0717 | Val Acc: 98.51%
Epoch [10/15] | Train Loss: 0.1640 | Train Acc: 96.12% | Val Loss: 0.0626 | Val Acc: 98.51%
Epoch [11/15] | Train Loss: 0.1161 | Train Acc: 96.95% | Val Loss: 0.0613 | Val Acc: 98.8

In [31]:
from prepare_data import create_dataloaders
train_loader, _, _ = create_dataloaders(batch_size=32)

# Get the class names
classes = train_loader.dataset.classes
print(classes)

# See one batch
inputs, labels = next(iter(train_loader))
print(labels)  # Tensor of integers like tensor([3, 5, 2, 7, ...])
print([classes[label] for label in labels])  # Converts back to building names


Datasets ready: 1212 train, 335 val, 205 test samples.
['Bahnhofstraße 7', 'Barfüßstraße 1', 'Biegenstraße 11', 'Biegenstraße 12', 'Biegenstraße 14', 'Biegenstraße 9', 'Deutschhausstraße 1', 'Deutschhausstraße 10, Deutches Haus, Fachbereich Geographie', 'Deutschhausstraße 12', 'Deutschhausstraße 17', 'Deutschhausstraße 17A', 'Deutschhausstraße 17B', 'Deutschhausstraße 3', 'Ketzerbach 11', 'Ketzerbach 63', 'Landgraf-Philipp-Straße 4', 'Marbacher Weg 6', 'Mineralogisches Museum', 'Pilgrimstein 2', 'Renthof 5', 'Renthof 6', 'Robert-Koch-Straße 4', 'Robert-Koch-Straße 6', 'Roter-Graben-10', 'Schulstraße 12', 'Universitätsstraße 24', 'Universitätsstraße 25', 'Universitätsstraße 6', 'Universitätsstraße 7']
tensor([15, 16, 28,  7, 14, 21,  5,  7, 15, 15,  7, 24, 26, 21, 25,  4, 12, 18,
         2, 25,  7,  4, 22,  8, 22, 16,  4, 27, 18,  3, 20, 28])
['Landgraf-Philipp-Straße 4', 'Marbacher Weg 6', 'Universitätsstraße 7', 'Deutschhausstraße 10, Deutches Haus, Fachbereich Geographie', 'Ketzerba

In [34]:
from torchvision import transforms
from PIL import Image
import torch

# Load your trained model
model = models.resnet50(weights=None)
model.fc = nn.Linear(model.fc.in_features, 29)  # Adjust for 29 classes
model.load_state_dict(torch.load('resnet50_marburglens.pth'))
model.eval()
model.to(DEVICE)

# Prepare the same transform as validation
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

# Load and preprocess your image
img_path = 'test_images/myphoto.jpg'  # <- your image path
image = Image.open(img_path).convert('RGB')
input_tensor = transform(image).unsqueeze(0).to(DEVICE)  # Add batch dimension

# Make prediction
with torch.no_grad():
    outputs = model(input_tensor)
    _, preds = torch.max(outputs, 1)

# Map prediction to building name
from prepare_data import create_dataloaders
train_loader, _, _ = create_dataloaders(batch_size=32)
class_names = train_loader.dataset.classes

predicted_class = class_names[preds.item()]
print(f'Predicted Building: {predicted_class}')


C:\Users\sOrOush\AppData\Local\Temp\ipykernel_95352\3207310441.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('resnet50_marburglens.pth

Datasets ready: 1212 train, 335 val, 205 test samples.
Predicted Building: Universitätsstraße 25
